# Simple Examples using Default Background Pixels

B.J. Rauscher<br>
NASA/GSFC

## 1 INTRODUCTION

**If you are able to specify which pixels can be treated as background, STOP. You will get better results using "enhanced MASK mode" as described in `enhanced_mask_example.ipynb`.**

This notebook describes how to use the first generation NIRSpec cleaner (NSClean1). The technique described here may be incrementally better than the NIRSpec Team's "rolling median" correction. We are leaving this notebook in the distribution because it may be helpful for "quick look" analysis or by those who do not have background pixel masks.

In [1]:
# Other standard imports
import os
import numpy as np
from astropy.io import fits
from time import perf_counter as t # Used for benchmarking
from glob import glob

# It will be handy to have a stack
stack = []

# Import the NIRSpec Clean package
import nsclean as nc
opmode = 'IFU' # Set this to 'MOS' or 'IFU'

NRS2 seems to have stronger "picture frame" noise and vertical banding than NRS1. Instantiate a NIRSpec cleaner object for it.

In [2]:
cleaner = nc.NSClean1(opmode, 'NRS2')

Read in some data. This should be a "slope image" like those written by the STScI pipeline. The spectral dispersion direction should be horizontal as displayed by SAOImage DS9.

In [3]:
# Make a list of input files. You won't have the same directories, so just pick
# pipeline Stage 1 rate image.
files = glob('/local/data/home/brausche/data1/JWST/Birkmann_IFU'+\
                 '/1270_rate/**/*_nrs2_rate.fits', recursive=True)

# Pick one
file = files[1]

# Read data. The STScI files put the data in hdul[1].
with fits.open(file) as hdul:
    D  = hdul[1].data
    H0 = hdul[0].header
    H1 = hdul[1].header

Clean D. On my server, this takes about TBD seconds. You can speed this up by enabling multithreading or using a GPU. The notebook `advanced_mask_example.ipynb` shows how to do both.

In [4]:
stack.append(t()) # Setup to benchmark
D = cleaner.clean(D) # This is the line that does all the work
print('Elapsed time (s) = ', t()-stack.pop()) # Benchmark it!

Elapsed time (s) =  55.605339932022616


Save results. The `chsuf()` utility changes the output filename's suffix from `.fits` to `.cln.fits`.

In [5]:
H0['comment'] = 'Processed by NSClean Rev. ' + nc.__version__
hdul = fits.HDUList([fits.PrimaryHDU(header=H0),
                   fits.ImageHDU(D, header=H1, name='SCI')])
hdul.writeto('/local/data/home/brausche/tmp/'+\
                 nc.chsuf(os.path.basename(file), '.cln.fits'),
                    overwrite=True)
hdul.close()